<a href="https://colab.research.google.com/github/Elliocharis/DAT-490/blob/main/DAT_490_Capstone_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steam Comment Scraping: https://partner.steamgames.com/doc/store/getreviews

Below is a work in progress. There appears to be a hiccup when I run it on a larger date range

In [68]:
import requests
import json
from urllib.parse import quote


appid = "413150" # Stardew Valley, will need to figure out how to get a list of all appids, maybe cross referenced from steamDB
filter = 'all'
language = 'english'
day_range = 30  # WIP, problems with higher numbers of days. confirm against steam review count (will be slightly higher due to including 'spam')
cursor = "*" # this enables us to keep pulling page after page by passing the cursor
review_type = 'all'
purchase_type = 'all' # default is Steam
num_per_page = 100 # max is 100
filter_offtopic_activity = 0 # Steam defaults to filtering out review bombs; I'm including them for now

all_reviews = []


def get_reviews(appid, cursor):
    cursor = quote(cursor)
    url = f"https://store.steampowered.com/appreviews/{appid}?json=1&filter={filter}&language={language}&day_range={day_range}&cursor={cursor}&review_type={review_type}&purchase_type={purchase_type}&num_per_page={num_per_page}"
    print("Fetching URL: ", url)
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

while True: # until we can't produce a valid url (aka cursor breaks or no more reviews)
    data = get_reviews(appid, cursor)
    if data is None or 'reviews' not in data or not data['reviews']:
        print("No more reviews or missing data in response.")
        break

    all_reviews.extend(data['reviews'])
    if 'query_summary' in data:
            print("Query Summary:", data['query_summary'])
            # Check if num_reviews is less than 100
            if data['query_summary'].get('num_reviews', 0) < 100:
                print("\nFewer than 100 reviews retrieved, breaking the loop.")
                break
    cursor = data['cursor']
    print(f"reviews retrieved: {len(all_reviews)}")

print(f"Total reviews retrieved: {len(all_reviews)}")

# Saving to Json, can download in files on the right
with open("test_reviews.json", "w") as f:
    json.dump(all_reviews, f, indent=4)
    print("Saved data to test_reviews.json")

Fetching URL:  https://store.steampowered.com/appreviews/413150?json=1&filter=all&language=english&day_range=30&cursor=%2A&review_type=all&purchase_type=all&num_per_page=100
Query Summary: {'num_reviews': 100, 'review_score': 9, 'review_score_desc': 'Overwhelmingly Positive', 'total_positive': 370076, 'total_negative': 4305, 'total_reviews': 374381}
reviews retrieved: 100
Fetching URL:  https://store.steampowered.com/appreviews/413150?json=1&filter=all&language=english&day_range=30&cursor=AoIFQFaAAAAAAABztPCQBQ%3D%3D&review_type=all&purchase_type=all&num_per_page=100
Query Summary: {'num_reviews': 100}
reviews retrieved: 200
Fetching URL:  https://store.steampowered.com/appreviews/413150?json=1&filter=all&language=english&day_range=30&cursor=AoIFQFaAAAAAAABwu8OPBQ%3D%3D&review_type=all&purchase_type=all&num_per_page=100
Query Summary: {'num_reviews': 100}
reviews retrieved: 300
Fetching URL:  https://store.steampowered.com/appreviews/413150?json=1&filter=all&language=english&day_range=